# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 

##  Final Project: Milestone 3 - Final Project [70 pts]


**Harvard University**<br/>
**Spring 2020**<br/>
**Group Members**: Fernando Medeiros, Mohammed Gufran Pathan, and Prerna Aggarwal<br/>

<hr style="height:2pt">

---

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

<div class="theme"> Final Deliverables </div>

1. **Code Report:** You are expected to submit the code you developed as part of the course project. The commented code should be provided in report format. This means that each group in a Jupyter notebook should explain—in a clean and concise report fashion—how they proceeded at every step and coding/methodology  choices . The code report should have a structure that consists of an introduction, body and conclusion.
1. **Ignite Talk:** You will present the talk on 5/11, 5/12, or 5/13. Details to come for Ignite Talk guidelines.

[Final Project Guidelines](https://docs.google.com/document/d/1Zhmm9JP4FGQBi5abFiM22e5iXYo_rr7i_vbpW0-xt8A/edit)

In [2]:
# Baisc Python Libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

plt.style.use("ggplot")

In [3]:
# Project Python Lybraries
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from pandas.io.json import json_normalize
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import json
import lxml
import lzma
import nltk
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.layers import Input, SimpleRNN, Embedding, Dense, TimeDistributed, GRU,\
                                    Dropout, Bidirectional, Conv1D, BatchNormalization
from tensorflow.keras.models import Model, Sequential,model_from_json
from tensorflow.keras.utils import to_categorical

print(tf.keras.__version__)
print(tf.__version__)

2.2.4-tf
2.0.0


In [4]:
# Set seed for repeatable results
np.random.seed(123)
tf.random.set_seed(456)

<div class="theme"> Part 1: Data Prep </div>
Read `north_carolina.xz` into a pandas dataframe using the provided code below.

In [5]:
#base_path = "./data/xml"
base_path = "./data/text"
state = 'north_carolina.xz'
f = lzma.open(os.path.join(base_path, state), "rb")
state_data = f.readlines()
f.close()

In [6]:
data_json = [json.loads(line) for line in state_data]
print(f'Flattening data for {state}')
data = json_normalize(data_json)

Flattening data for north_carolina.xz


In [7]:
# fix date
data['decision_date_p'] = pd.to_datetime(data.decision_date, errors='coerce')
# add year of decision date to dataset
data['decision_year'] = data.decision_date_p.dt.year
# add month of decision date to dataset
data['decision_month'] = data.decision_date_p.dt.month

In [8]:
# select one year of cases
data_2015 = data[(data.decision_year==2015) & (data.decision_month==1)]

In [9]:
# view top 5 cases
data_2015.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,cites_to,...,jurisdiction.name,jurisdiction.whitelisted,jurisdiction.id,jurisdiction.url,jurisdiction.slug,casebody.status,casebody.data,decision_date_p,decision_year,decision_month
92741,4329281,https://api.capapi.org/v1/cases/4329281/,State v. Robert McPhail,State v. McPhail,2015-01-22,341P14,811,811,"[{'type': 'official', 'cite': '367 N.C. 811'}]",[],...,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0,1.0
92758,4332768,https://api.capapi.org/v1/cases/4332768/,"Clorey Eugene France v. Glover & Petersen, P.A...","France v. Glover & Petersen, P.A.",2015-01-22,452P14,813,813,"[{'type': 'official', 'cite': '367 N.C. 813'}]",[],...,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0,1.0
92770,4325043,https://api.capapi.org/v1/cases/4325043/,State v. Joshua Andrew Stepp,State v. Stepp,2015-01-22,038A14,809,809,"[{'type': 'official', 'cite': '367 N.C. 809'}]",[],...,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0,1.0
92775,4323615,https://api.capapi.org/v1/cases/4323615/,State v. Raymond Dakim Harris Joiner,State v. Joiner,2015-01-22,514P13-3,814,814,"[{'type': 'official', 'cite': '367 N.C. 814'}]",[],...,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0,1.0
92827,4326314,https://api.capapi.org/v1/cases/4326314/,State v. Reginald U. Fullard,State v. Fullard,2015-01-22,475P14,814,814,"[{'type': 'official', 'cite': '367 N.C. 814'}]",[],...,N.C.,True,5,https://api.capapi.org/v1/jurisdictions/nc/,nc,ok,<?xml version='1.0' encoding='utf-8'?>\n<caseb...,2015-01-22,2015.0,1.0


In [10]:
data_2015[data_2015.id==4329281]['casebody.data'].iloc[0]

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody firstpage="811" lastpage="811" xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1">\n  <docketnumber id="b851-8">341P14</docketnumber>\n  <parties id="b851-9">State v. Robert McPhail</parties>\n  <opinion type="majority">\n    <p id="b851-10">Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)</p>\n    <p id="b851-11">Denied</p>\n    <judges id="b851-12">Ervin, J., recused</judges>\n  </opinion>\n</casebody>\n'

In [11]:
data_2015['casebody.data'].iloc[0]

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody firstpage="811" lastpage="811" xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1">\n  <docketnumber id="b851-8">341P14</docketnumber>\n  <parties id="b851-9">State v. Robert McPhail</parties>\n  <opinion type="majority">\n    <p id="b851-10">Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)</p>\n    <p id="b851-11">Denied</p>\n    <judges id="b851-12">Ervin, J., recused</judges>\n  </opinion>\n</casebody>\n'

In [12]:
BeautifulSoup(data_2015['casebody.data'].iloc[0], 'xml')

<?xml version="1.0" encoding="utf-8"?>
<casebody firstpage="811" lastpage="811" xmlns:="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1">
<docketnumber id="b851-8">341P14</docketnumber>
<parties id="b851-9">State v. Robert McPhail</parties>
<opinion type="majority">
<p id="b851-10">Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)</p>
<p id="b851-11">Denied</p>
<judges id="b851-12">Ervin, J., recused</judges>
</opinion>
</casebody>

In [13]:
BeautifulSoup(data_2015['casebody.data'].iloc[0], 'xml').find_all(text=True)

['\n',
 '341P14',
 '\n',
 'State v. Robert McPhail',
 '\n',
 '\n',
 'Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)',
 '\n',
 'Denied',
 '\n',
 'Ervin, J., recused',
 '\n',
 '\n']

In [14]:
BeautifulSoup(data_2015['casebody.data'].iloc[0], 'xml').headnotes

In [15]:
BeautifulSoup(data_2015['casebody.data'].iloc[0], 'xml').opinion.find_all(text=True)

['\n',
 'Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)',
 '\n',
 'Denied',
 '\n',
 'Ervin, J., recused',
 '\n']

In [16]:
data_2015['headnote.flag'] = 0
for i, xml in enumerate(data_2015['casebody.data']):
    if BeautifulSoup(xml, 'xml').headnotes:
        print(i)
        data_2015['headnote.flag'].iloc[i] = 1

data_2015[['id','headnote.flag']]

6
8
14


/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/share/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-

,id,headnote.flag
92741,4329281,0
92758,4332768,0
92770,4325043,0
92775,4323615,0
92827,4326314,0
92843,4321452,0
92853,4323320,1
92856,4321553,0
92874,4328336,1
92888,4318433,0


In [17]:
# case without headnotes
print(BeautifulSoup(data_2015[data_2015.id==4329281]['casebody.data'].iloc[0], 'xml').headnotes)

# casw with headnotes
print(BeautifulSoup(data_2015[data_2015.id==4323320]['casebody.data'].iloc[0], 'xml').headnotes)

None
<headnotes id="b781-8">1. Utilities — general rate case — changing economic conditions — impact on customers — findings</headnotes>


In [18]:
BeautifulSoup(data_2015[data_2015.id==4323320]['casebody.data'].iloc[0], 'xml')

<?xml version="1.0" encoding="utf-8"?>
<casebody firstpage="741" lastpage="752" xmlns:="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1">
<parties id="b781-5">STATE OF NORTH CAROLINA ex rel. UTILITIES COMMISSION, PUBLIC STAFF—NORTH CAROLINA UTILITIES COMMISSION, and DUKE ENERGY CAROLINAS, LLC v. ATTORNEY GENERAL ROY COOPER and NORTH CAROLINA WASTE AWARENESS AND REDUCTION NETWORK</parties>
<docketnumber id="b781-6">No. 12A14</docketnumber>
<decisiondate id="b781-7">(Filed 23 January 2015)</decisiondate>
<headnotes id="b781-8">1. Utilities — general rate case — changing economic conditions — impact on customers — findings</headnotes>
<headnotes id="b781-9">The Utilities Commission made sufficient findings regarding the impact of changing economic conditions upon customers in a general rate case and those findings were supported by competent, material, and substantial evidence in view of the entire record. The Commission’s findings not only demonstrated that the Comm

In [19]:
BeautifulSoup(data_2015[data_2015.id==4323320]['casebody.data'].iloc[0], 'xml').find_all('author')

[<author id="b782-11">JACKSON, Justice.</author>]

In [20]:
BeautifulSoup(data_2015[data_2015.id==4323320]['casebody.data'].iloc[0], 'xml').find_all('opinion')

[<opinion type="majority">
 <author id="b782-11">JACKSON, Justice.</author>
 <p id="b782-12">In this case we consider whether the order of the North Carolina Utilities Commission (“the Commission”) authorizing a 10.2% return on equity (“ROE”) for Duke Energy Carolinas (“Duke”) contained sufficient findings of fact to demonstrate that the order was supported by competent, material, and substantial evidence in view of the entire record. <em>See </em>N.C.G.S. § 62-94 (2013). In addition, we consider whether the Commission’s use of the single coincident peak (“1CP”) cost-of-service methodology unreasonably discriminated against residential <page-number citation-index="1" label="743">*743</page-number>customers and whether the Commission inappropriately shifted certain expenses to ratepayers. Because we conclude that the Commission made sufficient findings of fact regarding the impact of changing economic conditions upon customers, that the use of 1CP was supported by substantial evidence, 

In [21]:
[text for text in BeautifulSoup(data_2015[data_2015.id==4323320]['casebody.data'].iloc[0],
                                'xml').find_all(text=True) if text.parent.name != "headnotes"]

['\n',
 'STATE OF NORTH CAROLINA ex rel. UTILITIES COMMISSION, PUBLIC STAFF—NORTH CAROLINA UTILITIES COMMISSION, and DUKE ENERGY CAROLINAS, LLC v. ATTORNEY GENERAL ROY COOPER and NORTH CAROLINA WASTE AWARENESS AND REDUCTION NETWORK',
 '\n',
 'No. 12A14',
 '\n',
 '(Filed 23 January 2015)',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '*742',
 '\n',
 '\n',
 'On direct appeal as of right pursuant to N.C.G.S. §§ 7A-29(b) and 62-90(d) from a final order of the North Carolina Utilities Commission entered on 24 September 2013 in Docket No. E-7, Sub 1026. Heard in the Supreme Court on 8 September 2014.',
 '\n',
 '\n',
 'Troutman Sanders LLP, by Kiran H. Mehta; Heather Shirley Smith, Deputy General Counsel, and Charles A. Castle, Associate General Counsel, Duke Energy Carolinas, LLC; and Williams Mullen, by Christopher G. Browning, Jr., for applicant-appellee Duke Energy Carolinas, LLC.',
 '\n',
 '\n',
 'Antoinette R. Wike, Chief Counsel, and William E. Grantmyre, David T. Drooz, and Robert S. Gi